# Revisão OPENMP

### COMO COMPILAR CÓDIGO

```
g++ -O3 -fopenmp <file_name> -o <file_name>
```

### Comandos de configuração e identificação.

``` C++
omp_set_num_threads(int num_threads)    // Set num of available threads
omp_get_num_threads()                   // Get num of threads currently executing
omp_get_max_threads()                   // Get num of threads available for the program

omp_get_thread_num()                    // Get num of thread that is executing 
omp_get_num_procs()                     // Get num of processors available

omp_get_wtime()                         // return wall clock
omp_get_wtick()                         // resolution of clock
```

### Configuração de variáveis compartilhadas

``` C++
#pragma omp parallel private(...)       // Váriavel é única para cada thread
#pragma omp parallel shared(...)        // Váriavle é compartilhada com cada thread 
```

### Configuração de semáforos
``` C++ 
#pragma omp critical                    // Apenas uma thread pode processar esse bloco por vez
```

### Configurações de paralelização
``` C++
#pragma omp parallel                    // Bloco de operação paralela padrão
#pragma omp parallel for                // Paralelização otimizada para for loops
```

### Configurações de tasks e master
``` C++
#pragma omp task                        // Tarefas são agendadas para rodar em cada thread de um bloco
#pragma omp master                      // Tarefas "mestre" rodam apenas uma vez em na thread de índice 0
``` 

### Cálculo do PI com openMP

In [ ]:
%%writefile pi_omp_task.cpp
#include <omp.h>
#include <iostream>
#include <iomanip>
static long num_steps = 1024l*1024*1024*2;

#define MIN_BLK  1024*1024*256

double sum = 0;

void pi_r(long Nstart, long Nfinish, double step) {
    long i,iblk;
    #pragma omp parallel reduction(+ : sum)
    {
    if (Nfinish-Nstart < MIN_BLK){
        for (i = Nstart; i < Nfinish; i++){
            double x = (i+0.5)*step;
            sum += 4.0/(1.0+x*x); 
        }
    } else {
            #pragma omp master
            {
                iblk = Nfinish-Nstart;
                #pragma omp task
                pi_r(Nstart,         Nfinish-iblk/2,step);
                #pragma omp task
                pi_r(Nfinish-iblk/2, Nfinish,       step);
            }
        }
    }
}

int main () {
    double step, pi;
    double init_time, final_time;
    step = 1.0/(double) num_steps;
    init_time = omp_get_wtime();
    pi_r(0, num_steps, step);
    pi = step * sum;
    final_time = omp_get_wtime() - init_time;

    std::cout << "for " << num_steps << " steps pi = " << std::setprecision(15) << pi << " in " << final_time << " secs\n";
}
